In [2]:
import pandas as pd

df = pd.read_csv('dataset/synthetic_logs.csv')
df.head(10)

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert


In [26]:
df.drop("complexity", axis=1, inplace=True)

In [27]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [28]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [29]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

# Load the pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for log messages
embeddings = model.encode(df['log_message'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 76/76 [00:19<00:00,  3.98it/s]


In [30]:
# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

# Add the cluster labels to the DataFrame
df['cluster'] = clusters
df.head(10)

,timestamp,source,log_message,target_label,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,0,None
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,3,None
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,4,System Notification
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,5,None
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,0,None


In [31]:
df[df.cluster == 1].head(10)  # Show noise points

,timestamp,source,log_message,target_label,cluster,regex_label
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,1,None
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,1,None
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,1,None
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,1,None
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,1,None
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,1,None
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,1,None
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,1,None
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,1,None


In [32]:
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts > 10]

for cluster in large_clusters:
    print(f"Cluster {cluster}:")
    print(df[df['cluster'] == cluster]['log_message'].head(10).to_string(index=False))
    print("\n---\n")

Cluster 1017:
Series([], )

---

Cluster 147:
Series([], )

---

Cluster 100:
Mail server faced a transmission problem

---

Cluster 86:
Admin privilege misuse detected for user 3117

---

Cluster 60:
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...

---

Cluster 60:
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...

---

Cluster 58:
Server 36 faced an unanticipated shutdown durin...
Server 22 faced an unanticipated shutdown durin...
Server 3 faced an unanticipated shutdown during...

---

Cluster 57:
Server 18 went offline unexpectedly during data...
Server 28 went offline unexpectedly during data...
Server 19 went offline unexpectedly during data...

---

Cluster 53:
 Module X experienced an invalid data format issue
        Input format mismatch occurred in module X
Module X repo

In [33]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return label
    return None


In [34]:
classify_with_regex("User User494 logged OUT.")

'User Action'

In [35]:
classify_with_regex("Hello, this is a test message.")

In [36]:
df["regex_label"] = df["log_message"].apply(classify_with_regex)
df.head(10)

,timestamp,source,log_message,target_label,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,0,None
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,3,None
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,4,System Notification
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,5,None
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,0,None


In [37]:
df_non_regex = df[df["regex_label"].isnull()].copy()
df_non_regex.head(10)

,timestamp,source,log_message,target_label,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,0,None
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,0,None
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,3,None
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,5,None
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,0,None
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,1,None


In [38]:
df_non_regex.shape

(1910, 6)